In [3]:
# Load Libraries

import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio
import yaml

sys.path.append("../src/")

from geostax.utils import grid
from geostax.ingestion import read_data
from geostax.utils.grid import make_cuboidal_grid, make_topographic_grid
from geostax.utils.desurvey import bulk_trajectories, compute_trajectories, extract_xyz_assays
from geostax.topography.naive_gp import gp_topography, gp_topo_check

# Modify some package settings

pio.renderers.default = "browser"
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)
# plt.style.use(["ieee"])

In [4]:
# Load path configuration file

with open("../config/paths.yaml") as stream:
    try:
        paths = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [5]:
# Main

# Read data

df_collars, df_survey, df_assays = read_data(paths)

# Idiosyncratic preprocessing needed for collars dataset

df_collars["season_start"] = df_collars["Season"].str.split("/").str[0]  # .astype(int)
df_collars["season_start"] = df_collars["season_start"].astype(float)
df_collars["season_end"] = df_collars["Season"].str.split("/").str[1]  # .astype(int)
df_collars["season_end"] = df_collars["season_end"].astype(float)

# Holes of interest

holes_of_interest = df_collars["HOLE-ID"].to_list()

# Filter the survey data now

df_survey_filt = df_survey.copy()

# Compute trajectories of all drill holes

df_traj = bulk_trajectories(df_survey, df_collars, holes_of_interest)

# GP topography

z_inferred, grid_data = gp_topography(df_traj, df_collars, res=100)
fig = gp_topo_check(
    grid_data["x_grid_all"], 
    grid_data["y_grid_all"], 
    z_inferred, 
    grid_data["x_obs"], 
    grid_data["y_obs"], 
    grid_data["z_obs"]
)
fig.show()

# Grid creation

xyz_grid_r2, dz_implied, z_lowest_collar = make_cuboidal_grid(
    df_traj, 
    z_inferred, 
    grid_data["x_grid"], 
    grid_data["y_grid"],
    res=100
)

xyz_grid_r1 = make_topographic_grid(grid_data["xy_grid_all"], dz_implied, z_lowest_collar, z_inferred)
xyz_grid = np.vstack([xyz_grid_r1, xyz_grid_r2])
# xyz_grid = np.vstack([xyz_grid_r2])
with open("../cache/xyz_grid.npy", "wb") as f:
    np.save(f, xyz_grid)

# Assay assignment to x, y, z

df_assay_coords = extract_xyz_assays(df_traj, df_assays)
df_assay_coords.to_parquet("../cache/df_assay_coords.parquet")
df_assay_coords.to_csv("../cache/df_assay_coords.csv", index=False)

100%|██████████| 342/342 [00:00<00:00, 603.20it/s]


(1000000, 3)
(365484, 3)
